# Майнор ВШЭ
## Прикладные задачи анализа данных 2020
## Семинар 12: NER и лингвомодели

Cегодня поговорим об одной из самых популярных задач NLP – извлечении именованных сущностей (Named-entity recognition, NER).

![image.png](https://lh4.googleusercontent.com/proxy/44YAkWHn9Jv83xnTa3aXCGZLzBsiYRuz0Xji9E4kL_poSNCCsbwYpfqRDdlOAyJXGjevcO9KW-f2mKFfcBGSzc0Qq4vUUERyFGlF2Qhf583LrCVVHfHIg2MZ=w1200-h630-p-k-no-nu)

Первая задача, с решения которой начинается процесс работы каждой NLP системы, это необходимость максимально детального разбора пользовательского запроса и извлечения из него всей доступной в нем информации.  
  
NLP система должна:  
    1) Разбить текст запроса на составляющие (слова, токены)  
    2) Привести их в базовую форму (леммы, стеммы). Получить, если это необходимо, сведения о частях речи и прочую низкоуровневую информацию.  
    3) Распознать слова и словосочетания, то есть вычленить из текста именованные сущности и попытаться извлечь из них так называемые нормализованные значения.  

Этот процесс называется распознаванием именованных сущностей — NER, а модули, занимающиеся их извлечением — компонентами NER.

Допустим, есть новостной текст, и мы хотим выделить в нем сущности (некоторый заранее зафиксированный набор — например, персоны, локации, организации, даты и т.д.)   
Задача NER – понять, что участок текста “июль 1805 года” является датой, “Анна Павловна Шерер” – персоной и т.д.

На входе текст, на выходе структурированные, нормализованные объекты, например, с именами, адресами, датами.

![image.png](https://lh5.googleusercontent.com/proxy/1k-A3kOGEBbN28paVDWBTjtCEwZqfMcIlf_Xq-8TiLsaYpLDfVNYzi_-PMOaqR0wWeQuDnuef9H7q3FhgkwGySJC01OopKlcGyQ7gdAQhDR_P0J-LY_h6Nzx=w1200-h630-p-k-no-nu)

Что такое именованные сущности?  
В первой, классической постановке, которая была сформулирована на конференции MUC-6 в 1995 году, это персоны, локации и организации. С тех пор появилось несколько доступных корпусов, в каждом из которых свой набор именованных сущностей. Обычно к персонам, локациям и организациям добавляются новые типы сущностей.  
Самые распространенные из них — числовые (даты, денежные суммы), а также сущности Misc (от miscellaneous — прочие именованные сущности; пример — iPhone X ).

Для английского языка существует масса коммерческих и открытых решений: Spacy, Stanford NER, OpenNLP, NLTK, MITIE, Google Natural Language API, ParallelDots, Aylien, Rosette, TextRazor.  
Для русского тоже есть хорошие решения, но они в основном закрытые: DaData, Abbyy Infoextractor, Promt и др.
Есть неплохой открытый natasha и deeppavlov

Разберем на практике некоторые из них

## 1 Спарсим случайную статью для анализа

In [8]:
from os import path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

% matplotlib inline

In [9]:
!pip install beautifulsoup4

In [10]:
import re

import requests
from bs4 import BeautifulSoup

In [11]:
requests.get("http://rbc.ru/")

<Response [200]>

In [32]:
%%time
resp = requests.get("https://lenta.ru/news/2020/04/20/12/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie lid=vAsAAPBpoF4FeznCAVlfAAB= for .lenta.ru/>, <Cookie lids=48207B051A7C15B2 for .lenta.ru/>]>
time to download: 0:00:00.322966
page encoding utf-8
Server response:  200
Is everything ok?  True
Page's URL:  https://lenta.ru/news/2020/04/20/12/
CPU times: user 14.7 ms, sys: 7.96 ms, total: 22.7 ms
Wall time: 355 ms


In [33]:
resp.text[:1000]

'<!DOCTYPE html><html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head><title>Раскрыт внешний вид iPhone 12: Гаджеты: Наука и техника: Lenta.ru</title><meta content="Раскрыт внешний вид iPhone 12: Гаджеты: Наука и техника: Lenta.ru" name="title" /><meta content="text/html; charset=utf-8" http-equiv="Content-Type" />\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"66a8d51230","applicationID":"1241738","transactionName":"J19cQUoOWA0ERBoQXhRZUUYXElwOFg==","queueTime":0,"applicationTime":156,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQUGU1VRGwICUFBVBAk=",licenseKey:"66a8d51230",applicationID:"1241738"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("fun

In [34]:
BeautifulSoup(resp.text, "html5lib").get_text()

'Раскрыт внешний вид iPhone 12: Гаджеты: Наука и техника: Lenta.ru\nwindow.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"66a8d51230","applicationID":"1241738","transactionName":"J19cQUoOWA0ERBoQXhRZUUYXElwOFg==","queueTime":0,"applicationTime":156,"agent":""}\n(window.NREUM||(NREUM={})).loader_config={xpid:"VQUGU1VRGwICUFBVBAk=",licenseKey:"66a8d51230",applicationID:"1241738"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(18),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_d

In [35]:
bs = BeautifulSoup(resp.text, "html5lib")
title = bs.h1.text
text = BeautifulSoup(
    " ".join([p.text for p in bs.find_all("p")]), "html5lib"
).get_text()
print(title, "\n-----\n", text)

Раскрыт внешний вид iPhone 12 
-----
 Следующий iPhone выйдет с новым дисплеем и не получит поддержки стилуса. Об этом сообщил известный инсайдер Джон Проссер (Jon Prosser) в своем Twitter. В серии постов Проссер раскрыл полученную от источников информацию о внешнем виде смартфонов серии iPhone 12. В частности, он опубликовал схему дисплея новых iPhone, согласно которой аппарат получит уменьшенный экранный вырез. «Монобровь» устройства будет иметь меньший размер за счет измененной компоновки камер и сенсоров.  По информации Проссера, iPhone 12 выйдет с разъемом Lightning, которым комплектуются все смартфоны компании, начиная с выпущенного в 2012 году iPhone 5. Таким образом, эксперт опроверг слухи о том, что новые смартфоны по аналогии с актуальными iPad Pro будут комплектоваться портом USB-C. Инсайдер также подтвердил, что девайсы серии iPhone 12 не будут иметь смарт-коннектора для крепления аксессуаров и поддержку стилуса. Судя по ранним слухам, новый iPhone должен получить несколько

## 2 Natasha

Наташа — это аналог Томита-парсера для Питона (Yargy-парсер) плюс набор готовых правил для извлечения имён, адресов, дат, сумм денег и других сущностей.

Сейчас в Наташе есть правила для извлечения имён, адресов, дат и сумм денег.  
Есть ещё правила для названий организаций и географических объектов, но у них не очень высокое качество.

In [4]:
!pip install natasha

     |████████████████████████████████| 777 kB 28 kB/s eta 0:00:01
     |████████████████████████████████| 41 kB 442 kB/s eta 0:00:011
     |████████████████████████████████| 46 kB 1.2 MB/s eta 0:00:011
     |████████████████████████████████| 7.1 MB 209 kB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=19851 sha256=da678c8844212479e056c4dd080c8f93facdd55c2ff54c3b480dd2c92b070c60
  Stored in directory: /root/.cache/pip/wheels/63/74/c4/1e4f0884a7ba7c26baa4a841240a507953dcfe7de606522571
Successfully built docopt


### 2.1 Имена

In [28]:
from natasha import NamesExtractor
from natasha.markup import show_json, show_markup

extractor = NamesExtractor()

matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
show_json(facts)

На юго-западе Москвы загорелся восьмиэтажный жилой дом, говорится в сообщении столичного управления МЧС в Twitter. Пожар произошел по адресу Ленинский проспект дом 62/1. Сообщение о возгорании поступило в 02:57. По информации ведомства, горят квартира на восьмом этаже и чердак. Общая площадь возгорания достигла двух тысяч квадратных метров. В МЧС сообщили, что при тушении удалось спасти четырех человек. Данных о пострадавших нет. Как сообщили ТАСС источники в экстренных службах, из дома эвакуировали 120 человек. Один из них находился на карантине в связи с коронавирусом. В ведомстве также сообщили, что распространению пожара способствуют деревянные перекрытия и пустоты. Пожару присвоен третий ранг сложности из пяти возможных. Вечером 8 апреля в Москве в доме престарелых «Третий возраст» произошло задымление. Здание загорелось на улице 3-я Мякининская, 50 человек были эвакуированы, еще 13 были спасены сотрудниками МЧС. Возгорание произошло на площади 15 квадратных метров на минус первом

В 2016 году проходило соревнование factRuEval-2016 по извлечению именованных сущностей. Среди участников были крупные компании: ABBYY, RCO.  
У топовых решений F1-мера для имён была 0.9+. У Наташи результат хуже — 0.78. Проблема в основном c иностранными именами и сложными фамилиями, например: «Харуки Мураками», "… главой Афганистана Хамидом Карзаем", «Остап Бендер встречается с Кисой Воробьяниновым ...»  Для текстов с русскими именами качество получается ~0.95.


#### Задание 1

In [1]:
### Найти статью с русскими именами и вывести имена из статьи

### 2.2 Адреса

Интерфейс такой же как для имён, только NamesExtractor меняется на AddressExtractor:

In [37]:
resp = requests.get("https://lenta.ru/news/2020/04/13/gorim/")

bs = BeautifulSoup(resp.text, "html5lib")
title = bs.h1.text
text = BeautifulSoup(
    " ".join([p.text for p in bs.find_all("p")]), "html5lib"
).get_text()
print(title, "\n-----\n", text)

В Москве загорелся жилой дом 
-----
 На юго-западе Москвы загорелся восьмиэтажный жилой дом, говорится в сообщении столичного управления МЧС в Twitter. Пожар произошел по адресу Ленинский проспект дом 62/1. Сообщение о возгорании поступило в 02:57. По информации ведомства, горят квартира на восьмом этаже и чердак. Общая площадь возгорания достигла двух тысяч квадратных метров. В МЧС сообщили, что при тушении удалось спасти четырех человек. Данных о пострадавших нет. Как сообщили ТАСС источники в экстренных службах, из дома эвакуировали 120 человек. Один из них находился на карантине в связи с коронавирусом. В ведомстве также сообщили, что распространению пожара способствуют деревянные перекрытия и пустоты. Пожару присвоен третий ранг сложности из пяти возможных. Вечером 8 апреля в Москве в доме престарелых «Третий возраст» произошло задымление. Здание загорелось на улице 3-я Мякининская, 50 человек были эвакуированы, еще 13 были спасены сотрудниками МЧС. Возгорание произошло на площади

In [38]:
from natasha import AddressExtractor
from natasha.markup import show_json, show_markup

extractor = AddressExtractor()

matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
show_json(facts)

На юго-западе Москвы загорелся восьмиэтажный жилой дом, говорится в сообщении столичного управления МЧС в Twitter. Пожар произошел по адресу [[Ленинский проспект дом 62/1]]. Сообщение о возгорании поступило в 02:57. По информации ведомства, горят квартира на восьмом этаже и чердак. Общая площадь возгорания достигла двух тысяч квадратных метров. В МЧС сообщили, что при тушении удалось спасти четырех человек. Данных о пострадавших нет. Как сообщили ТАСС источники в экстренных службах, из дома эвакуировали 120 человек. Один из них находился на карантине в связи с коронавирусом. В ведомстве также сообщили, что распространению пожара способствуют деревянные перекрытия и пустоты. Пожару присвоен третий ранг сложности из пяти возможных. Вечером 8 апреля в Москве в доме престарелых «Третий возраст» произошло задымление. Здание загорелось на [[улице 3-я Мякининская, 50]] человек были эвакуированы, еще 13 были спасены сотрудниками МЧС. Возгорание произошло на площади 15 квадратных метров на мину

In [39]:
extractor(text)

Matches('На юго-западе Москвы загорелся восьмиэтажный жилой дом, говорится в сообщении столичного управления МЧС в Twitter. Пожар произошел по адресу Ленинский проспект дом 62/1. Сообщение о возгорании поступило в 02:57. По информации ведомства, горят квартира на восьмом этаже и чердак. Общая площадь возгорания достигла двух тысяч квадратных метров. В МЧС сообщили, что при тушении удалось спасти четырех человек. Данных о пострадавших нет. Как сообщили ТАСС источники в экстренных службах, из дома эвакуировали 120 человек. Один из них находился на карантине в связи с коронавирусом. В ведомстве также сообщили, что распространению пожара способствуют деревянные перекрытия и пустоты. Пожару присвоен третий ранг сложности из пяти возможных. Вечером 8 апреля в Москве в доме престарелых «Третий возраст» произошло задымление. Здание загорелось на улице 3-я Мякининская, 50 человек были эвакуированы, еще 13 были спасены сотрудниками МЧС. Возгорание произошло на площади 15 квадратных метров на минус первом и подвальном этажах. Жертвами пожара стали пять человек.',
        [Match([MorphTagToken('Ленинский',
                              [141, 150),
                              'RU',
                              'I',
                              [Form('ленинский', Grams(ADJF,masc,nomn,sing)),
                               Form('ленинский',
                                    Grams(ADJF,accs,inan,masc,sing))]),
                MorphTagToken('проспект',
                              [151, 159),
                              'RU',
                              'O',
                              [Form('проспект',
                                    Grams(NOUN,inan,masc,nomn,sing)),
                               Form('проспект',
                                    Grams(NOUN,accs,inan,masc,sing))]),
                MorphTagToken('дом',
                              [160, 163),
                              'RU',
                              'O',
                              [Form('дом', Grams(NOUN,accs,inan,masc,sing)),
                               Form('дом', Grams(NOUN,inan,masc,nomn,sing))]),
                TagToken('62', [164, 166), 'INT', 'O'),
                TagToken('/', [166, 167), 'PUNCT', 'O'),
                TagToken('1', [167, 168), 'INT', 'O')],
               [141, 168)),
         Match([MorphTagToken('улице',
                              [841, 846),
                              'RU',
                              'O',
                              [Form('улица', Grams(NOUN,femn,inan,loct,sing)),
                               Form('улица',
                                    Grams(NOUN,datv,femn,inan,sing))]),
                TagToken('3', [847, 848), 'INT', 'I'),
                TagToken('-', [848, 849), 'PUNCT', 'I'),
                MorphTagToken('я',
                              [849, 850),
                              'RU',
                              'I',
                              [Form('я', Grams(1per,NPRO,nomn,sing))]),
                MorphTagToken('Мякининская',
                              [851, 862),
                              'RU',
                              'I',
                              [Form('мякининский',
                                    Grams(ADJF,femn,nomn,sing))]),
                TagToken(',', [862, 863), 'PUNCT', 'O'),
                TagToken('50', [864, 866), 'INT', 'O')],
               [841, 866))])

В factRuEval-2016 участникам предлагалось извлекать имена, названия организаций и географические объекты.  
Независимых тестовых данных для оценки качества работы с адресами, на сколько я знаю, не существует. За несколько лет работы у нас накопились сотни тысяч строк вида «Адрес: 443041 г. Самара ул. Ленинская, д.168», «Адрес г. Иркутск, ул. Байкальская, д. 133, офис 1 (вход со двора).». Для оценки качества сделана случайная выборка из 1000 адресов, результаты проверены вручную, ~90% строк обработались корректно. Проблемы возникают, в основном, с названиями улиц, например: «г. Волжск, 2-ая промышленная, стр. 2», «111674, г. Москва, Дмитриевского, д. 17».

### 2.3 Другие правила

Ещё в Наташе есть правила для дат и денег. Интерфейс такой же как у AddressExtractor и NamesExtractor.  
*DatesExtractor*  
*MoneyExtractor*

#### Задание 2

In [ ]:
### Найти и загрузить статью с датами и вывести даты из статьи

## 3 SpaCy -- NER

Использование библиотеки SpaCy

SpaCy — это open-source библиотека для NLP, написанная на Python и Cython. В отличие от NLTK, который широко используется для преподавания и исследований, spaCy фокусируется на предоставлении программного обеспечения для разработки.

In [34]:
!pip install spacy

Удобная инструкция для инсталляции на   
https://spacy.io/usage

In [0]:
import spacy

# Загружаем модель
nlp = spacy.load("en_core_web_sm")

# Передаем текст в модель
doc = nlp("i love coffee")

### Некоторые основные возможности NLP в библиотеке spaCy

#### Токенизация

Текст необходимо токенизировать и spaCy позволяет нам это сделать.  
> _"Given a character sequence and a defined document unit, tokenization is the task of chopping it up into pieces, called tokens."_

(words, sentences, punctuation, etc.)

Подробности реализации здесь:
https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html

В spaCy многие конструкции обозначаются тегами, например теги nsubj, prt, RB и др.  
Полную расщифровку можно получить с помощью explain()

In [69]:
spacy.explain("nsubj")

'nominal subject'

Полное описание возможностей есть на странице spaCy
https://spacy.io/api/annotation

#### Часть речи (Part Of Speech POS)

POS для английского языка: noun, verb, adjective, adverb, pronoun, preposition, conjunction, interjection, numeral, article, determiner

https://en.wikipedia.org/wiki/Part_of_speech

#### Синтаксическая зависимость (Syntactic dependency DEP)

DEP показывает связи между токенами

"I" это _nominal subject_ и "coffee" это _direct object_.  Они соединены с помощью _root_ "love".

#### Нарисуем визуализацию

Построим дерево зависимостей с помощью displaCy 

In [61]:
from spacy import displacy

# Outside of jupyter, use displacy.serve()
displacy.render(doc, style="dep", jupyter=True)

#### Задание 3

In [0]:
### Придумайте более сложное предложение и постройте синтаксичекую зависимость для него


Если передаем в dос предложение без именованных сущностей (организации, имена и тп), то при попытке вывести сущности получим Warning

In [65]:
displacy.render(doc, style="ent", jupyter=True)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [66]:
doc.ents

()

Возьмем предложение с именованными сущностями

In [0]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

In [71]:
displacy.render(doc, style="dep", jupyter=True)

In [72]:
displacy.render(doc, style="ent", jupyter=True)

### Доступ к атрбутам токенов

In [43]:
print(doc[0])
print(doc[0].text)

Apple
Apple


In [44]:
for token in doc:
    print(token)

Apple
is
looking
at
buying
U.K.
startup
for
$
1
billion


In [45]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple PROPN nsubj
is AUX aux
looking VERB ROOT
at ADP prep
buying VERB pcomp
U.K. PROPN compound
startup NOUN dobj
for ADP prep
$ SYM quantmod
1 NUM compound
billion NUM pobj


In [46]:
print(
    f'{"text":<10} {"part of speech":<15} {"description":<15} {"dependency":<15} {"description":<30} {"lemma":<20} {"is stop":<10}'
)

for token in doc:
    print(
        f"{token.text:<10} {token.pos_:<15} {str(spacy.explain(token.pos_)):<15} {token.dep_:<15} {str(spacy.explain(token.dep_)):<30} {token.lemma_:<20} {token.is_stop:<10}"
    )

text       part of speech  description     dependency      description                    lemma                is stop   
Apple      PROPN           proper noun     nsubj           nominal subject                Apple                0         
is         AUX             auxiliary       aux             auxiliary                      be                   1         
looking    VERB            verb            ROOT            None                           look                 0         
at         ADP             adposition      prep            prepositional modifier         at                   1         
buying     VERB            verb            pcomp           complement of preposition      buy                  0         
U.K.       PROPN           proper noun     compound        compound                       U.K.                 0         
startup    NOUN            noun            dobj            direct object                  startup              0         
for        ADP          

#### Lemma

Лемма слова.

Например: "be" основа "am", "are" и "is"

https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html#2378

#### Стоп слова

Стоп-слова -- это слова, которые часто удаляются из текста перед последующим анализом.

Подробнее:
https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py

Стоп-слова можно посмотреть здесь `nlp.Defaults.stop_words`

In [47]:
print(nlp.Defaults.stop_words)

# Можно добавлять и удалять слова из списка
# https://spacy.io/usage/v2#migrating-saving-loading

# nlp.Defaults.stop_words.add("across")
# nlp.Defaults.stop_words.remove("above")

{'therefore', 'hence', 'hereupon', 'fifty', 'thus', 'no', 'please', 'used', 'into', 'nobody', 'or', 'wherein', 'while', 'still', 'somewhere', 'every', 'herein', 'him', 'ours', 'himself', 'beyond', 'themselves', 'however', 'and', 'own', 'under', '‘ve', 'onto', 'as', 'could', 'very', 'below', 'these', 'around', 'call', 'being', 'against', 'the', 'up', 'back', 'last', 'most', 'would', 'whereupon', 'off', 'so', 'how', 'regarding', 'without', 'he', 'unless', 'n‘t', 'somehow', 'before', 'more', 'after', '‘s', 'front', '’s', 'both', 'down', 'by', 'same', 'anything', 'which', 'sometimes', 'with', 'yourselves', 'during', 'hers', 'all', 'full', 'amount', '’ve', '’ll', 'at', 'whoever', 'our', 'get', 'afterwards', 'even', 'among', 'four', 'other', 'noone', 'take', 'say', 'because', 'became', 'latterly', 'over', 'throughout', 'who', 'former', 'did', 'nine', 'this', 'above', 'since', 'moreover', 'have', 'can', 'another', 'they', 'make', 'though', 'were', 'will', 'yet', 'that', 'about', 'something', 

# Сохранение и загрузка модели

Сохранение модели

```python
some_dir = '/path/to/directory' 
nlp.to_disk(some_dir)
```

Загрузка модели

```python
nlp = spacy.load(some_dir)
```

# Поиск именованных сущностей в тексте

Источник: https://www.techinasia.com/yellow-messenger-banks-20m

Используем large модель ('en_core_web_lg') чтобы проанализировать статью.

In [0]:
text = """
Global demand for conversational AI systems has become urgent and vast. According to research from advisory firm Gartner, one in every three companies worldwide has already integrated the technology into their operations.

India-based Yellow Messenger, a startup that provides businesses with an AI chatbot for customer engagement, said it has raised US$20 million in series B money to capture more of the growing market.

The new funding round, which was led by Lightspeed Venture Partners and Lightspeed India, bumps up the company’s total funding to US$24 million, according to a statement.

Yellow Messenger looks to use the fresh capital to fuel international growth across Latin America and Asia Pacific, as well as new regions such as the US and Europe.

In addition, it plans to invest further in product innovation, which includes improving its bot’s multilingual voice capabilities and expanding enterprise integrations. Currently, its chatbots are available in over 120 languages, including Bahasa, Bengali, Cantonese, English, Hindi, and Thai.

The startup is also looking to launch a developer marketplace for virtual assistants, which will enable third-party developers to create, launch, and distribute their own chatbots to Yellow Messenger clients.

Through a partnership with Facebook Messenger, the company said it has recently been appointed by the National Health Authority of India to provide state governments with chatbots to help with citizen engagement amid the Covid-19 pandemic.

Raghu Ravinutala, CEO and co-founder of Yellow Messenger, said that at this time, the startup has seen a surge in global demand from the banking, healthcare, food and beverage, retail, and ecommerce sectors as a result of the crisis. In response, the company has deployed special chatbots for enterprises tailored to drive business continuity.

Yellow Messenger was founded in 2017. It claims to have been growing 5x in bookings year on year since its establishment. It has over 100 clients globally, including companies such as Accenture, Flipkart, Grab, and Xiaomi India.
"""

In [0]:
# !python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [50]:
doc = nlp(text)
print(doc)


Global demand for conversational AI systems has become urgent and vast. According to research from advisory firm Gartner, one in every three companies worldwide has already integrated the technology into their operations.

India-based Yellow Messenger, a startup that provides businesses with an AI chatbot for customer engagement, said it has raised US$20 million in series B money to capture more of the growing market.

The new funding round, which was led by Lightspeed Venture Partners and Lightspeed India, bumps up the company’s total funding to US$24 million, according to a statement.

Yellow Messenger looks to use the fresh capital to fuel international growth across Latin America and Asia Pacific, as well as new regions such as the US and Europe.

In addition, it plans to invest further in product innovation, which includes improving its bot’s multilingual voice capabilities and expanding enterprise integrations. Currently, its chatbots are available in over 120 languages, includi

### Выведем предложения

In [51]:
for sent in doc.sents:
    print(sent)


Global demand for conversational AI systems has become urgent and vast.
According to research from advisory firm Gartner, one in every three companies worldwide has already integrated the technology into their operations.


India-based Yellow Messenger, a startup that provides businesses with an AI chatbot for customer engagement, said it has raised US$20 million in series B money to capture more of the growing market.


The new funding round, which was led by Lightspeed Venture Partners and Lightspeed India, bumps up the company’s total funding to US$24 million, according to a statement.


Yellow Messenger looks to use the fresh capital to fuel international growth across Latin America and Asia Pacific, as well as new regions such as the US and Europe.


In addition, it plans to invest further in product innovation, which includes improving its bot’s multilingual voice capabilities and expanding enterprise integrations.
Currently, its chatbots are available in over 120 languages, inc

In [52]:
sents_l = list(doc.sents)
print(sents_l[0])
print(sents_l[0][0])
print(sents_l[0][0].pos_)


Global demand for conversational AI systems has become urgent and vast.


SPACE


### Визуализация сущностей в статье

In [53]:
displacy.render(doc, style="ent", jupyter=True)

### Описание сущностей

In [54]:
print(f'{"entity":<50} {"label":<10} description')
for ent in doc.ents:
    print(f"{ent.text:<50} {ent.label_:<10} {spacy.explain(ent.label_)}")

entity                                             label      description
Gartner                                            ORG        Companies, agencies, institutions, etc.
three                                              CARDINAL   Numerals that do not fall under another type
India                                              GPE        Countries, cities, states
Yellow Messenger                                   PRODUCT    Objects, vehicles, foods, etc. (not services)
US$20 million                                      MONEY      Monetary values, including unit
Lightspeed Venture Partners                        ORG        Companies, agencies, institutions, etc.
Lightspeed India                                   ORG        Companies, agencies, institutions, etc.
US$24 million                                      MONEY      Monetary values, including unit
Yellow Messenger                                   PRODUCT    Objects, vehicles, foods, etc. (not services)
Latin America        

# Дообучение модели

Если точности нашшей модели недостаточно, можем дообучить предобученную модель.  
Это позволит нам не использовать огромный датасет для обучения.

Подробнее и по шагам описано здесь: https://spacy.io/usage/training

In [55]:
!python3 -m spacy download en_core_web_sm
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

     |████████████████████████████████| 12.0MB 1.1MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011742 sha256=ddefa3cd2b263f4c152888420a53bd890c9afcf2f2642e4e9a8376d233dc9611
  Stored in directory: /tmp/pip-ephem-wheel-cache-av1vhqal/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 12.0MB 1.7MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-cp36-none-any.whl size=12019125 sha256=8be4943407415b2949d9bcfa052f881834c5c91ab2fab063eec49a231ca4ff06
  Stored in directory: /root/.cache/pip/wheels/48/5c/1c/15f9d02afc8221a668d2172446dd8467b20cdb9aef80a172a4
Successfully built en-core-web-sm
  Fou

In [56]:
import random
from pathlib import Path

from spacy.util import compounding, minibatch

# данные для обучения
TRAIN_DATA = [
    (
        "Yellow Messenger is a startup in India that provides businesses with an AI chatbot.",
        {"entities": [(0, 16, "ORG"), (33, 38, "GPE")]},
    ),
    (
        "The co-founder of Yellow Messenger said some things.",
        {"entities": [(18, 33, "ORG")]},
    ),
    (
        "Facebook developed the service Facebook Messenger.",
        {"entities": [(0, 8, "ORG"), (31, 49, "PRODUCT")]},
    ),
]


def train_ner(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)


new_model_dir = "/"
train_ner(model="en_core_web_lg", output_dir=new_model_dir, n_iter=100)

Loaded model 'en_core_web_lg'
Losses {'ner': 11.224060987238772}
Losses {'ner': 7.2387266876176}
Losses {'ner': 8.016034741791373}
Losses {'ner': 6.681522947619669}
Losses {'ner': 6.096581724792486}
Losses {'ner': 7.0157991744108585}
Losses {'ner': 8.253963147210015}
Losses {'ner': 6.052718233200721}
Losses {'ner': 8.18513622134924}
Losses {'ner': 6.349734542891383}
Losses {'ner': 6.418410168495029}
Losses {'ner': 5.932170374318957}
Losses {'ner': 9.195402521640062}
Losses {'ner': 10.464522568508983}
Losses {'ner': 9.115126617252827}
Losses {'ner': 8.098959363996983}
Losses {'ner': 3.8592554461210966}
Losses {'ner': 6.749124456313439}
Losses {'ner': 3.9973622940742644}
Losses {'ner': 1.2819379150969326}
Losses {'ner': 7.789909660816193}
Losses {'ner': 5.482485926557274}
Losses {'ner': 3.991020564455539}
Losses {'ner': 4.076320219784975}
Losses {'ner': 4.101875526481308}
Losses {'ner': 7.869909346103668}
Losses {'ner': 5.178065974265337}
Losses {'ner': 4.221163452457404}
Losses {'ner': 

In [0]:
text = """
Global demand for conversational AI systems has become urgent and vast. According to research from advisory firm Gartner, one in every three companies worldwide has already integrated the technology into their operations.

India-based Yellow Messenger, a startup that provides businesses with an AI chatbot for customer engagement, said it has raised US$20 million in series B money to capture more of the growing market.

The new funding round, which was led by Lightspeed Venture Partners and Lightspeed India, bumps up the company’s total funding to US$24 million, according to a statement.

Yellow Messenger looks to use the fresh capital to fuel international growth across Latin America and Asia Pacific, as well as new regions such as the US and Europe.

In addition, it plans to invest further in product innovation, which includes improving its bot’s multilingual voice capabilities and expanding enterprise integrations. Currently, its chatbots are available in over 120 languages, including Bahasa, Bengali, Cantonese, English, Hindi, and Thai.

The startup is also looking to launch a developer marketplace for virtual assistants, which will enable third-party developers to create, launch, and distribute their own chatbots to Yellow Messenger clients.

Through a partnership with Facebook Messenger, the company said it has recently been appointed by the National Health Authority of India to provide state governments with chatbots to help with citizen engagement amid the Covid-19 pandemic.

Raghu Ravinutala, CEO and co-founder of Yellow Messenger, said that at this time, the startup has seen a surge in global demand from the banking, healthcare, food and beverage, retail, and ecommerce sectors as a result of the crisis. In response, the company has deployed special chatbots for enterprises tailored to drive business continuity.

Yellow Messenger was founded in 2017. It claims to have been growing 5x in bookings year on year since its establishment. It has over 100 clients globally, including companies such as Accenture, Flipkart, Grab, and Xiaomi India.
"""

In [58]:
nlp = spacy.load(new_model_dir)

doc = nlp(text)

displacy.render(doc, style="ent", jupyter=True)

# Результаты

Обучение помогло модели распознать корректно _Yellow Messenger_ как компанию и _Facebook Messenger_ как продукт.
Иногда это может приводить к переобучению и ваши предыдущие сущности могут отображаться некорректно. В таких случаях spaCy позволяет также создавать отдельно жесткие правила под необходимые сущности.


#### Задание 4

Найдите текст на английском с неразмеченной сущностью и дообучите вашу модель, разметив в обучении эту сущность.